In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/car-child-seat-sales-b1/sample_submission.csv
/kaggle/input/car-child-seat-sales-b1/train.csv
/kaggle/input/car-child-seat-sales-b1/test.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/car-child-seat-sales-b1/train.csv")
test_data = pd.read_csv("/kaggle/input/car-child-seat-sales-b1/test.csv")

In [3]:
train_data["Category"] = pd.cut(train_data["totalSales"], bins=[-float("inf"), 5, 10, float("inf")], labels=[1, 2, 3])
target = train_data["Category"]
train_data = train_data.drop(["ID", "totalSales", "Category"], axis=1)

In [4]:
categorical_cols = ["Location", "Urban", "US"]
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    train_data[col] = label_encoders[col].fit_transform(train_data[col])
    test_data[col] = label_encoders[col].transform(test_data[col])
train_data

,CompetitorPrice,Income,Advertising,Population,Price,Location,Age,Education,Urban,US
0,131,29,11,335,127,0,33,12,1,1
1,148,51,16,148,150,2,58,17,0,1
2,118,71,12,44,89,2,67,18,0,1
3,86,54,0,497,64,0,33,12,1,0
4,113,66,16,322,74,1,76,15,1,1
...,...,...,...,...,...,...,...,...,...,...
344,138,61,12,156,120,2,25,14,1,1
345,130,58,19,366,139,0,33,16,1,1
346,144,69,11,131,104,2,47,11,1,1
347,129,76,16,58,121,2,69,12,1,1


In [5]:
X_train, X_val, y_train, y_val = train_test_split(train_data, target, test_size=0.47, random_state=42)

In [6]:
logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [7]:
val_predictions = logreg.predict(X_val)

In [8]:
accuracy = (val_predictions == y_val).mean()
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.6787878787878788


In [9]:
logreg_final = LogisticRegression(max_iter=10000)
logreg_final.fit(train_data, target)

LogisticRegression(max_iter=10000)

In [10]:
test_data["Category"] = logreg_final.predict(test_data.drop("ID", axis=1))

In [11]:
submission = test_data[["ID", "Category"]]
submission.to_csv("submission.csv", index=False)